# Modelo Predicción de Ocupación
### Ejecución de modelo entrenado

## Contenido
* Parámetros generales e Importacion de biblioticas
* Carga de los datos
* Parámetros Generales
* Preprocesamiento
 * Unificación 
* Modelo
 * Procesamiento
 * Tidy Table
 * Separación Xvar y Yvar  (número variable de periodo)
 * Separacion Train y Test
 * Configuración de los Modelos
 * Funciones 
  * F. para evaluar varios modelos
  * F. para categorizar en semáforo
  * F. Evaluación de resultados Exactitud
* Generacion de Pronosticos a futuro
 * Procesamiento entrada para pronóstico
 * Ejecución del modelos y archivo de entrega

## Parámetros generales e Importacion de biblioticas

In [71]:
%load_ext autoreload
%autoreload 2
np.set_printoptions(suppress=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from datetime import datetime
#from datetime import timedelta
from dateutil.relativedelta import relativedelta
#import datetime
from dateutil.relativedelta import relativedelta
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from xgboost import plot_importance
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix # es una manera para resumir los datos 
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve #roc_curve
from sklearn.metrics import roc_auc_score #auc
from joblib import dump,load

## Carga de los datos

In [3]:
data_ocupacionNodo = pd.read_csv('../DatosOrigen/OCUPACION_NODO.csv', sep=';')
data_NodoINFI = pd.read_csv('../DatosOrigen/NODO_INFI.csv', sep=';')
data_SEG = pd.read_csv('../DatosOrigen/segmentacion.csv', sep=';') 

### Base data_NodoINF

In [5]:
data_NodoINFI['DIA']=1 # le agrego la columna dia
data_NodoINFI['MES'] = [f'0{x}' if x <10 else str(x) for x in data_NodoINFI['MES']]
data_NodoINFI['DIA'] = [f'0{x}' if x <10 else str(x) for x in data_NodoINFI['DIA']]
data_NodoINFI['FECHA'] = (2000+data_NodoINFI['ANIO']).astype('str') + '-' + data_NodoINFI['MES'] + '-' + data_NodoINFI['DIA']
# eliminamos columnas sobrantes
data_NodoINFI=data_NodoINFI.drop(['DIA','MES','ANIO'], axis=1)
# # renombramos columnas
data_NodoINFI.rename(columns={'NODO':'COD_NODO'}, inplace=True)
### completar Nullos con 0
data_NodoINFI=data_NodoINFI.fillna(0)

### Base data_NodoINF data_SEG

In [6]:
data_SEG.drop(['NUEVA ESTRUCTURA'], axis=1,inplace=True)
data_SEG['SEGMENTADO']=1

## Preprocesamiento
### Unificacion

In [7]:
data_ocupacion=pd.merge(data_ocupacionNodo, data_NodoINFI, on = ('COD_NODO','FECHA'), how='left')
data_ocupacion=pd.merge(data_ocupacion, data_SEG, on = ('COD_NODO','FECHA'), how='left')

In [9]:
data_ocupacion.head(2)

,COD_NODO,FECHA,OCUPACION_DW,OCUPACION_UP,CANT_USER,PROM_VEL,MEDIANA_VEL,VEL_ME_8,VEL_10_25,VEL_30,...,VEL_MAX,VEL_MIN,RENTA,PROM_RENTA,CONVERGENTE,SERV_INT,SERV_TV,SERV_VOZ,CANT_CALL,SEGMENTADO
0,PKE,2020-06-01,0.355112,0.124435,90.0,17.0,10.0,32.0,31.0,16.0,...,50.0,3.0,7925605.0,88062.0,22.0,90.0,83.0,83.0,4.0,NaN
1,2AU3,2020-06-01,0.208285,0.242637,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Parámetros generales

In [10]:
start = datetime.now()

In [11]:
fechaMaxima=datetime.strptime(data_ocupacion['FECHA'].max(),'%Y-%m-%d')
periodos_y=7
periodos_x=7
variable_prediccion='OCUPACION_DW'

In [12]:
fechaLimite=fechaMaxima + relativedelta(months=-1*periodos_y)
fechaInicial=fechaLimite + relativedelta(months=-1*periodos_x)

### Selección Fechas

In [13]:
## Eliminar informaición anterior Junion 2022
data_ocupacion['FECHA']=data_ocupacion['FECHA'].astype('datetime64[ns]')
### Elininar datos últimos mes
data_ocupacion=data_ocupacion[data_ocupacion['FECHA']>=fechaInicial]

##  Funciones

In [14]:
#Funcion para Calcular el número de periodos hacia atrás
def diff_month(d1, d2):
    return (d1.year - d2.dt.year) * 12 + d1.month - d2.dt.month

In [15]:
def semaforo(df, col):
    df = df.copy()
    df.loc[(df[col]>=0.58) & (df[col]<=0.75), 'ESTATUS'] = 'AMARILLO'
    df.loc[df[col]>0.75, 'ESTATUS'] = 'ROJO'
    df.loc[df[col]<0.60, 'ESTATUS'] = 'VERDE'
    return(df)

In [16]:
def semaforoVal(val):
    #df = df.copy()
    respuesta='V'
    if (val>=0.58) & (val<=0.75):
        respuesta='A'
    elif val>0.75:
        respuesta='R'
    return(respuesta)

In [17]:
def semaforoNum2Val(val):   
    respuesta='V'
    if val==1:
        respuesta='A'
    elif val==2:
        respuesta='R'
    return(respuesta)

In [18]:
def semaforoVal2Num(val):   
    respuesta=0
    if val=='A':
        respuesta=1
    elif val=='R':
        respuesta=2
    return(respuesta)

In [19]:
def semaforoRojo(val):    
    if val>0.75:
        respuesta=1
    else :
        respuesta=0
    return(respuesta)

In [20]:
def semaforoAmarillo(val):    
    if (val>=0.58) & (val<=0.75):
        respuesta=1
    else :
        respuesta=0
    return(respuesta)

In [21]:
def semaforoRojoAmarillo(val):    
    if val>=0.58:
        respuesta=1
    else :
        respuesta=0
    return(respuesta)

In [22]:
def semaforoNum(val):
    #df = df.copy()
    respuesta=0
    if (val>=0.58) & (val<=0.75):
        respuesta=1
    elif val>0.75:
        respuesta=2
    return(respuesta)

In [23]:
def semaforoNum2Rojo(val):    
    respuesta=0
    if val==2:
        respuesta=1    
    return(respuesta)

In [24]:
def semaforoNum2Amarillo(val):    
    respuesta=0
    if val==1:
        respuesta=1    
    return(respuesta)

In [25]:
def semaforoNum2RojoAmarillo(val):    
    respuesta=0
    if val>=1:
        respuesta=1    
    return(respuesta)

In [26]:
def medidasEvaluacion(matrizConfucion):
    exactitud=np.trace(matrizConfucion)/matrizConfucion.sum()
    print ('exactitud:', exactitud)
    sensibilidad=[0]*3
    especificidad=[0]*3
    for i in range(0,len(matrizConfucion)):
        sensibilidad[i]=matrizConfucion[i,i]/matrizConfucion.sum(axis=1)[i]
        especificidad[i]=np.delete(np.delete(matrizConfucion,i, axis=0),i,axis=1).sum()/np.delete(matrizConfucion,i, axis=0).sum()
        print ('sensibilidad ',i,": ",sensibilidad[i])
        print ('especificidad ',i,": ",especificidad[i])

In [27]:
def sampling_strategy(X,y,n_samples,t='majority'):
    target_classes=''
    if t== 'majority':
        target_classes= y.value_counts() >n_samples
    elif t== 'minority':
        target_classes = y.value_counts() < n_samples
    tc=target_classes[target_classes ==True].index
    #target_classes_all=y.value_counts().index
    sampling_strategy={}
    for target in tc:
        sampling_strategy[target]= n_samples
    return sampling_strategy

In [28]:
def df2Feature_importanceClass(x_train_b,mod,title):
    #shap_test = x_train_b.sample(1000)
    shap_test = x_train_b
    shap_values = shap.TreeExplainer(mod).shap_values(shap_test)
    plt.figure()
    plt.title(title)
    shap.summary_plot(shap_values, shap_test,max_display=30)
    plt.subplots_adjust(left=0.35, right=0.9, top=0.9, bottom=0.3)    
    plt.show();

## Modelo
### Procesamiento

In [29]:
data_ocupacion['mesAnt']=(periodos_x+periodos_y)-diff_month(fechaMaxima,data_ocupacion['FECHA'])
### Ordenar por fecha
data_ocupacion=data_ocupacion.sort_values(["FECHA","COD_NODO"], ascending = (True, True))

### Seleccion Nodos para el Modelo

In [30]:
### Solo Nodos con información completa
seleccion=data_ocupacion[['COD_NODO',variable_prediccion]].groupby('COD_NODO').count()
seleccion[seleccion[variable_prediccion]>=periodos_y+periodos_x+1]
seleccion.rename(columns={variable_prediccion:'noRegVarPre'}, inplace=True)
data_ocupacion=data_ocupacion.merge(seleccion, on='COD_NODO', how='left')
data_ocupacion=data_ocupacion[data_ocupacion['noRegVarPre']>=periodos_y+periodos_x]
data_ocupacion.drop(columns='noRegVarPre', inplace=True)
### Nodos No Segmentados en el periodo de tiempo considerado
data_ocupacion=data_ocupacion[data_ocupacion['SEGMENTADO']!=1]
data_ocupacion.drop(columns='SEGMENTADO', inplace=True)
### Eliminar datos con nullos
data_ocupacion=data_ocupacion.dropna()

###  Selección Columnas

In [31]:
selCols=['COD_NODO','FECHA','OCUPACION_DW','OCUPACION_UP','SERV_INT','mesAnt']
data_ocupacionC=data_ocupacion.copy()
data_ocupacion=data_ocupacion[selCols]

### Tidy Tables

#### Regresión

In [32]:
cols=data_ocupacion.columns
cols=cols[2:-1]

In [33]:
cols_x=[]
for i, col in enumerate(cols):
    tablaAnalisis = pd.pivot_table(data_ocupacion.reset_index(), index='COD_NODO', columns='mesAnt', values=col)
    tablaAnalisis.rename(columns={x: (col+"-"+str(x)) for x in tablaAnalisis.columns}, inplace=True)
    #Identificacion Columnas X y Y    
    cols_x=cols_x+[(col+"-"+str(x)) for x in range(0,periodos_x)]    
    if i==0:
        tablaAnalisisTotal = tablaAnalisis
    else:
        tablaAnalisisTotal = tablaAnalisisTotal.merge(tablaAnalisis, how='outer', on='COD_NODO')

In [34]:
cols_x=sorted(list(set(cols_x)& set(tablaAnalisisTotal.columns.tolist())))

In [35]:
cols_y=[(variable_prediccion+"-"+str(x)) for x in range(periodos_x,periodos_x+periodos_y-1)]

In [36]:
yColClass=[s + '_sem2' for s in cols_y]

In [37]:
tablaAnalisisTotal=tablaAnalisisTotal.dropna()

In [38]:
data_ocupacionC.head()

,COD_NODO,FECHA,OCUPACION_DW,OCUPACION_UP,CANT_USER,PROM_VEL,MEDIANA_VEL,VEL_ME_8,VEL_10_25,VEL_30,...,VEL_MAX,VEL_MIN,RENTA,PROM_RENTA,CONVERGENTE,SERV_INT,SERV_TV,SERV_VOZ,CANT_CALL,mesAnt
0,01J,2020-07-01,0.316241,0.193338,27.0,38.0,30.0,3.0,4.0,11.0,...,100.0,5.0,2876249.0,106528.0,4.0,27.0,21.0,18.0,0.0,0
1,01S,2020-07-01,0.220020,0.217132,96.0,35.0,30.0,22.0,4.0,33.0,...,150.0,1.0,9072745.0,94508.0,54.0,96.0,73.0,79.0,12.0,0
3,02J,2020-07-01,0.316241,0.193338,13.0,37.0,30.0,0.0,4.0,3.0,...,100.0,10.0,1733201.0,133323.0,4.0,13.0,10.0,12.0,0.0,0
4,02S,2020-07-01,0.220020,0.167162,77.0,29.0,30.0,15.0,12.0,24.0,...,100.0,5.0,7199207.0,93496.0,30.0,77.0,62.0,69.0,5.0,0
5,03J,2020-07-01,0.316241,0.193338,24.0,30.0,30.0,2.0,7.0,7.0,...,75.0,3.0,2752809.0,114700.0,8.0,24.0,23.0,21.0,0.0,0


#### Clasificación

In [39]:
colsC=data_ocupacionC.columns
colsC=colsC[2:-1]

In [40]:
cols_xC=[]
for i, col in enumerate(colsC):
    tablaAnalisis = pd.pivot_table(data_ocupacionC.reset_index(), index='COD_NODO', columns='mesAnt', values=col)
    tablaAnalisis.rename(columns={x: (col+"-"+str(x)) for x in tablaAnalisis.columns}, inplace=True)
    #Identificacion Columnas X y Y    
    cols_xC=cols_xC+[(col+"-"+str(x)) for x in range(0,periodos_x)]    
    if i==0:
        tablaAnalisisTotalC = tablaAnalisis
    else:
        tablaAnalisisTotalC = tablaAnalisisTotalC.merge(tablaAnalisis, how='outer', on='COD_NODO')
tablaAnalisisTotalC=tablaAnalisisTotalC.dropna()

In [41]:
cols_xC=sorted(list(set(cols_xC)& set(tablaAnalisisTotalC.columns.tolist())))

## Carga de los modelos

In [42]:
ruta_objeto = f"../ModelosGenerados/modeloR.pkl"
modeloR = load(ruta_objeto)

In [43]:
ruta_objeto = f"../ModelosGenerados/modeloC.pkl"
modeloC = load(ruta_objeto)

### Selección datos Base

In [44]:
data_ocupacionNp=data_ocupacion.copy()
data_ocupacionNpC=data_ocupacionC.copy()

In [45]:
periodos_x=6
fechaMaxima=data_ocupacionNp['FECHA'].max()
fechaInicialNp=fechaMaxima + relativedelta(months=-1*periodos_x)

In [46]:
fechaMaxima

Timestamp('2021-09-01 00:00:00')

In [132]:
#meses a predecir
date_list = [(fechaMaxima + relativedelta(months=+x+1)).strftime('%b-%y') for x in range(6)]

In [133]:
date_list

['Oct-21', 'Nov-21', 'Dec-21', 'Jan-22', 'Feb-22', 'Mar-22']

In [47]:
data_ocupacionNp=data_ocupacionNp[data_ocupacionNp['FECHA']>=fechaInicialNp]
data_ocupacionNpC=data_ocupacionNpC[data_ocupacionNpC['FECHA']>=fechaInicialNp]
data_ocupacionNp['mesAnt']=(periodos_x)-diff_month(fechaMaxima,data_ocupacionNp['FECHA'])
data_ocupacionNpC['mesAnt']=(periodos_x)-diff_month(fechaMaxima,data_ocupacionNpC['FECHA'])

In [48]:
colsNp=data_ocupacionNp.columns
colsNp=colsNp[2:-1]
colsNpC=data_ocupacionNpC.columns
colsNpC=colsNpC[2:-1]

In [49]:
#Tabla para regresión
cols_xNp=[]
for i, col in enumerate(colsNp):
    tablaAnalisisNp = pd.pivot_table(data_ocupacionNp.reset_index(), index='COD_NODO', columns='mesAnt', values=col)
    tablaAnalisisNp.rename(columns={x: (col+"-"+str(x)) for x in tablaAnalisisNp.columns}, inplace=True)    
    cols_xNp=cols_xNp+[(col+"-"+str(x)) for x in range(0,periodos_x+1)]    
    if i==0:
        tablaAnalisisTotalNp = tablaAnalisisNp
    else:
        tablaAnalisisTotalNp = tablaAnalisisTotalNp.merge(tablaAnalisisNp, how='outer', on='COD_NODO')

In [50]:
#Tabla para clasificación
cols_xNpC=[]
for i, col in enumerate(colsNpC):
    tablaAnalisisNpC = pd.pivot_table(data_ocupacionNpC.reset_index(), index='COD_NODO', columns='mesAnt', values=col)
    tablaAnalisisNpC.rename(columns={x: (col+"-"+str(x)) for x in tablaAnalisisNpC.columns}, inplace=True)    
    cols_xNpC=cols_xNpC+[(col+"-"+str(x)) for x in range(0,periodos_x+1)]    
    if i==0:
        tablaAnalisisTotalNpC = tablaAnalisisNpC
    else:
        tablaAnalisisTotalNpC = tablaAnalisisTotalNpC.merge(tablaAnalisisNpC, how='outer', on='COD_NODO')

In [51]:
tablaAnalisisTotalNp=tablaAnalisisTotalNp.dropna()
tablaAnalisisTotalNpC=tablaAnalisisTotalNpC.dropna()

In [52]:
tablaResultados=tablaAnalisisTotalNp.copy()

In [53]:
#for predVar in modelos:
for predVar in cols_y:
    print(predVar)
    tablaResultados[predVar+'_pR']=modeloR[predVar].predict(tablaAnalisisTotalNp[cols_x])
    tablaResultados[predVar+'_pR_Sem']=tablaResultados[predVar+'_pR'].apply(semaforoVal)  

OCUPACION_DW-7
OCUPACION_DW-8
OCUPACION_DW-9
OCUPACION_DW-10
OCUPACION_DW-11
OCUPACION_DW-12


In [84]:
#for predVar in modelosClass:
for predVar in yColClass: 
    print(predVar)
    tablaResultados[predVar+'_pC']=modeloC[predVar].predict(tablaAnalisisTotalNpC[cols_xC])
    tablaResultados[predVar+'_pC_probR']=modeloC[predVar].predict_proba(tablaAnalisisTotalNpC[cols_xC])[:,2]
    tablaResultados[predVar+'_pC_probA']=modeloC[predVar].predict_proba(tablaAnalisisTotalNpC[cols_xC])[:,1]
    tablaResultados[predVar+'_pC']=tablaResultados[predVar+'_pC'].apply(semaforoNum2Val)    

OCUPACION_DW-7_sem2
OCUPACION_DW-8_sem2
OCUPACION_DW-9_sem2
OCUPACION_DW-10_sem2
OCUPACION_DW-11_sem2
OCUPACION_DW-12_sem2


In [89]:
import pandas as pd
pd.set_option('display.max_columns', None)
tablaResultados.head(10)

mesAnt,OCUPACION_DW-0,OCUPACION_DW-1,OCUPACION_DW-2,OCUPACION_DW-3,OCUPACION_DW-4,OCUPACION_DW-5,OCUPACION_DW-6,OCUPACION_UP-0,OCUPACION_UP-1,OCUPACION_UP-2,OCUPACION_UP-3,OCUPACION_UP-4,OCUPACION_UP-5,OCUPACION_UP-6,SERV_INT-0,SERV_INT-1,SERV_INT-2,SERV_INT-3,SERV_INT-4,SERV_INT-5,SERV_INT-6,OCUPACION_DW-7_pR,OCUPACION_DW-7_pR_Sem,OCUPACION_DW-8_pR,OCUPACION_DW-8_pR_Sem,OCUPACION_DW-9_pR,OCUPACION_DW-9_pR_Sem,OCUPACION_DW-10_pR,OCUPACION_DW-10_pR_Sem,OCUPACION_DW-11_pR,OCUPACION_DW-11_pR_Sem,OCUPACION_DW-12_pR,OCUPACION_DW-12_pR_Sem,OCUPACION_DW-7_sem2_pC,OCUPACION_DW-8_sem2_pC,OCUPACION_DW-9_sem2_pC,OCUPACION_DW-10_sem2_pC,OCUPACION_DW-11_sem2_pC,OCUPACION_DW-12_sem2_pC,OCUPACION_DW-7v1,OCUPACION_DW-7v2,OCUPACION_DW-7_Rank,OCUPACION_DW-8v1,OCUPACION_DW-8v2,OCUPACION_DW-8_Rank,OCUPACION_DW-9v1,OCUPACION_DW-9v2,OCUPACION_DW-9_Rank,OCUPACION_DW-10v1,OCUPACION_DW-10v2,OCUPACION_DW-10_Rank,OCUPACION_DW-11v1,OCUPACION_DW-11v2,OCUPACION_DW-11_Rank,OCUPACION_DW-12v1,OCUPACION_DW-12v2,OCUPACION_DW-12_Rank,OCUPACION_DW-7_sem2_pC_probR,OCUPACION_DW-7_sem2_pC_probA,OCUPACION_DW-8_sem2_pC_probR,OCUPACION_DW-8_sem2_pC_probA,OCUPACION_DW-9_sem2_pC_probR,OCUPACION_DW-9_sem2_pC_probA,OCUPACION_DW-10_sem2_pC_probR,OCUPACION_DW-10_sem2_pC_probA,OCUPACION_DW-11_sem2_pC_probR,OCUPACION_DW-11_sem2_pC_probA,OCUPACION_DW-12_sem2_pC_probR,OCUPACION_DW-12_sem2_pC_probA
COD_NODO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01J,0.302408,0.363124,0.362382,0.337471,0.350570,0.314207,0.332251,0.216276,0.221213,0.211418,0.220097,0.233507,0.257879,0.232684,24.0,26.0,26.0,25.0,24.0,23.0,24.0,0.355097,V,0.345597,V,0.372178,V,0.396380,V,0.392613,V,0.357224,V,V,V,V,V,V,V,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.000889,0.003046,0.001101,0.001976,0.001095,0.004972,0.001715,0.004285,0.001824,0.006895,0.013116,0.019625
01S,0.347061,0.401564,0.373240,0.348979,0.432986,0.404191,0.379371,0.298426,0.290753,0.339117,0.306406,0.269256,0.287102,0.310938,160.0,167.0,171.0,170.0,177.0,175.0,186.0,0.381625,V,0.401957,V,0.385453,V,0.461764,V,0.437555,V,0.446724,V,V,V,V,V,V,V,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.001848,0.012765,0.002072,0.007514,0.005878,0.092005,0.009508,0.064719,0.009118,0.081492,0.029886,0.025474
02J,0.302408,0.363124,0.362382,0.337471,0.350570,0.314207,0.332251,0.216276,0.221213,0.211418,0.220097,0.233507,0.257879,0.232684,12.0,12.0,13.0,14.0,15.0,13.0,12.0,0.353911,V,0.344182,V,0.369542,V,0.395421,V,0.392437,V,0.369531,V,V,V,V,V,V,V,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.001539,0.005309,0.002413,0.004422,0.002316,0.015015,0.004977,0.012138,0.004439,0.019927,0.036510,0.087421
02S,0.347061,0.401564,0.373240,0.348979,0.432986,0.404191,0.379371,0.286283,0.300031,0.301526,0.243914,0.227977,0.228819,0.247392,138.0,146.0,148.0,144.0,148.0,162.0,162.0,0.376773,V,0.394832,V,0.375707,V,0.451384,V,0.427671,V,0.448551,V,V,V,V,V,V,V,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.001382,0.007081,0.001097,0.003468,0.004217,0.074277,0.004951,0.026455,0.003147,0.041516,0.038980,0.016969
03J,0.302408,0.363124,0.362382,0.337471,0.350570,0.314207,0.332251,0.216276,0.221213,0.211418,0.220097,0.233507,0.257879,0.232684,32.0,32.0,32.0,32.0,30.0,29.0,29.0,0.356074,V,0.347116,V,0.371860,V,0.396233,V,0.392770,V,0.370169,V,V,V,V,V,V,V,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.000889,0.002957,0.001100,0.002563,0.001064,0.004737,0.001721,0.003741,0.001796,0.006308,0.014235,0.017049
04J,0.302408,0.363124,0.362382,0.337471,0.350570,0.314207,0.332251,0.216276,0.221213,0.211418,0.220097,0.233507,0.257879,0.232684,20.0,19.0,19.0,21.0,20.0,20.0,20.0,0.354601,V,0.344430,V,0.370709,V,0.396404,V,0.393534,V,0.369077,V,V,V,V,V,V,V,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.001651,0.005701,0.002671,0.006954,0.002190,0.013019,0.003474,0.008873,0.003962,0.014282,0.023026,0.066319
05J,0.283512,0.299967,0.302036,0.318871,0.289796,0.289193,0.260464,0.320807,0.306752,0.274566,0.257956,0.237106,0.280441,0.264702,61.0,60.0,60.0,6

####  Ranking de Nodos

In [57]:
reglasRank = [[0 , 0.2, 0.6],[0.2 , 0.4, 0.8],[0.6, 0.8, 1]]

In [58]:
def valRank(predR,predC):    
    return reglasRank[predR][predC]

In [59]:
#for predVar in modelos:    
for predVar in cols_y:
    tablaResultados[predVar+'v1']=tablaResultados[predVar+'_pR_Sem'].apply(semaforoVal2Num)
    tablaResultados[predVar+'v2']=tablaResultados[predVar+'_sem2_pC'].apply(semaforoVal2Num)    
    tablaResultados[predVar+'_Rank'] = tablaResultados.apply(lambda x: valRank(x[predVar+'v1'],x[predVar+'v2']), axis=1)  

In [92]:
## ordernar columnas
columnasResultados=[]
for predVar in cols_y:
    auxCol=[predVar+'_pR',predVar+'_pR_Sem',predVar+'_sem2_pC_probR',predVar+'_sem2_pC_probA',predVar+'_sem2_pC',predVar+'_Rank']
    columnasResultados=columnasResultados+auxCol

In [93]:
tablaResultados[columnasResultados].head(3)

mesAnt,OCUPACION_DW-7_pR,OCUPACION_DW-7_pR_Sem,OCUPACION_DW-7_sem2_pC_probR,OCUPACION_DW-7_sem2_pC_probA,OCUPACION_DW-7_sem2_pC,OCUPACION_DW-7_Rank,OCUPACION_DW-8_pR,OCUPACION_DW-8_pR_Sem,OCUPACION_DW-8_sem2_pC_probR,OCUPACION_DW-8_sem2_pC_probA,OCUPACION_DW-8_sem2_pC,OCUPACION_DW-8_Rank,OCUPACION_DW-9_pR,OCUPACION_DW-9_pR_Sem,OCUPACION_DW-9_sem2_pC_probR,OCUPACION_DW-9_sem2_pC_probA,OCUPACION_DW-9_sem2_pC,OCUPACION_DW-9_Rank,OCUPACION_DW-10_pR,OCUPACION_DW-10_pR_Sem,OCUPACION_DW-10_sem2_pC_probR,OCUPACION_DW-10_sem2_pC_probA,OCUPACION_DW-10_sem2_pC,OCUPACION_DW-10_Rank,OCUPACION_DW-11_pR,OCUPACION_DW-11_pR_Sem,OCUPACION_DW-11_sem2_pC_probR,OCUPACION_DW-11_sem2_pC_probA,OCUPACION_DW-11_sem2_pC,OCUPACION_DW-11_Rank,OCUPACION_DW-12_pR,OCUPACION_DW-12_pR_Sem,OCUPACION_DW-12_sem2_pC_probR,OCUPACION_DW-12_sem2_pC_probA,OCUPACION_DW-12_sem2_pC,OCUPACION_DW-12_Rank
COD_NODO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01J,0.355097,V,0.000889,0.003046,V,0.0,0.345597,V,0.001101,0.001976,V,0.0,0.372178,V,0.001095,0.004972,V,0.0,0.396380,V,0.001715,0.004285,V,0.0,0.392613,V,0.001824,0.006895,V,0.0,0.357224,V,0.013116,0.019625,V,0.0
01S,0.381625,V,0.001848,0.012765,V,0.0,0.401957,V,0.002072,0.007514,V,0.0,0.385453,V,0.005878,0.092005,V,0.0,0.461764,V,0.009508,0.064719,V,0.0,0.437555,V,0.009118,0.081492,V,0.0,0.446724,V,0.029886,0.025474,V,0.0
02J,0.353911,V,0.001539,0.005309,V,0.0,0.344182,V,0.002413,0.004422,V,0.0,0.369542,V,0.002316,0.015015,V,0.0,0.395421,V,0.004977,0.012138,V,0.0,0.392437,V,0.004439,0.019927,V,0.0,0.369531,V,0.036510,0.087421,V,0.0


In [138]:
nuevasColumnas=[]
for col in columnasResultados:
    n_col=col
    i=0
    for predVar in cols_y:
        n_col=n_col.replace(predVar,date_list[i])
        i=i+1
    nuevasColumnas.append(n_col)

In [140]:
tablaResultados=tablaResultados[columnasResultados]
tablaResultados.columns=nuevasColumnas

In [141]:
tablaResultados.head(3)

,Oct-21_pR,Oct-21_pR_Sem,Oct-21_sem2_pC_probR,Oct-21_sem2_pC_probA,Oct-21_sem2_pC,Oct-21_Rank,Nov-21_pR,Nov-21_pR_Sem,Nov-21_sem2_pC_probR,Nov-21_sem2_pC_probA,Nov-21_sem2_pC,Nov-21_Rank,Dec-21_pR,Dec-21_pR_Sem,Dec-21_sem2_pC_probR,Dec-21_sem2_pC_probA,Dec-21_sem2_pC,Dec-21_Rank,Jan-22_pR,Jan-22_pR_Sem,Jan-22_sem2_pC_probR,Jan-22_sem2_pC_probA,Jan-22_sem2_pC,Jan-22_Rank,Feb-22_pR,Feb-22_pR_Sem,Feb-22_sem2_pC_probR,Feb-22_sem2_pC_probA,Feb-22_sem2_pC,Feb-22_Rank,Mar-22_pR,Mar-22_pR_Sem,Mar-22_sem2_pC_probR,Mar-22_sem2_pC_probA,Mar-22_sem2_pC,Mar-22_Rank
COD_NODO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01J,0.355097,V,0.000889,0.003046,V,0.0,0.345597,V,0.001101,0.001976,V,0.0,0.372178,V,0.001095,0.004972,V,0.0,0.396380,V,0.001715,0.004285,V,0.0,0.392613,V,0.001824,0.006895,V,0.0,0.357224,V,0.013116,0.019625,V,0.0
01S,0.381625,V,0.001848,0.012765,V,0.0,0.401957,V,0.002072,0.007514,V,0.0,0.385453,V,0.005878,0.092005,V,0.0,0.461764,V,0.009508,0.064719,V,0.0,0.437555,V,0.009118,0.081492,V,0.0,0.446724,V,0.029886,0.025474,V,0.0
02J,0.353911,V,0.001539,0.005309,V,0.0,0.344182,V,0.002413,0.004422,V,0.0,0.369542,V,0.002316,0.015015,V,0.0,0.395421,V,0.004977,0.012138,V,0.0,0.392437,V,0.004439,0.019927,V,0.0,0.369531,V,0.036510,0.087421,V,0.0


In [142]:
tablaResultados.to_csv('Resultados/resultadosVp.csv', index=True,sep=';')

In [102]:
end = datetime.now()
print(f'Tiempo: {end - start} segundos')

Tiempo: 0:25:39.637837 segundos
